In [59]:
# naming convention
# Rid_Pop_program.txt
# e.g. SORCS3_YOL_program.txt?
# num indiv of pop by num of SNPs (-1, 0, or 1)

# data
import pandas as pd
import numpy as np
from scipy.linalg import solve_triangular
import json
    
# will make several projection functions that should all produce the same number to check.

def getProjection(y, X): # i thought this was the projection of X onto col but it's the other way around
    y = np.matrix(y)
    X = np.matrix(X)
    #print('\nX:-----\n', X)
    #print('\ny shape:-----\n', y.shape)
#     if y.shape[0] == 1: # this is bc sometimes if a single col is passed in as col, it becomes 1byx instead of xby1
#         y = y.T
    #print('\nnew y shape and y:-----\n', y.shape)
    # print('\n', y)
    # print(X.T.dot(X))
    #the projection of the matrix E on the subspace spanned by the i-th column of E
    inv = np.linalg.inv(X.T.dot(X))
    beta_est = inv.dot(X.T.dot(y)) # this is the full thing, projection of y onto X, 1x999
    #print("\nbeta shape:", beta_est.shape)
    return X.dot(beta_est)


def getProjectionPrintouts(y, X): # i thought this was the projection of X onto col but it's the other way around
    y = np.matrix(y)
    X = np.matrix(X)
    ###print('\nX:-----\n', X)
    #print('\ny shape:-----\n', y.shape)
#     if y.shape[0] == 1: # this is bc sometimes if a single col is passed in as col, it becomes 1byx instead of xby1
#         y = y.T
    ###print('\nnew y shape and y:-----\n', y.shape)
    # print('\n', y)
    # print(X.T.dot(X))
    #the projection of the matrix E on the subspace spanned by the i-th column of E
    inv = np.linalg.inv(X.T.dot(X))
    beta_est = inv.dot(X.T.dot(y))
    ###print("\nbeta shape:", beta_est.shape)
    return X.dot(beta_est)
    
    #beta = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(col))
    # ask if this is right (for lstsq stuff)
#     Q, R = np.linalg.qr(X)
#     X_proj = Q.T.dot(X)
#     return X_proj
    
def getProjection2(y, E):
    #the projection of the matrix E on the subspace spanned the i-th column of E
    Q, R = np.linalg.qr(E)
    # use R to solve for beta
    # R is upper triangular, so can use triangular solver:
    beta = solve_triangular(R, Q.T.dot(y))
    return beta
    # So now we have beta, and we can project the points using Q.T very simply:
#     X_proj = Q.T.dot(E)
#     return X_proj

def getProjection3(col, E):
    #the projection of the matrix E on the subspace spanned the i-th column of E
    return np.linalg.lstsq(E, col)
    
def getMagnitudeSq(x):
    #https://stackoverflow.com/questions/9171158/how-do-you-get-the-magnitude-of-a-vector-in-numpy
    #return np.linalg.norm(x)
    return x.dot(x)

def roundToInt(x):
    res = int(round(x))
    if res > 1:
        return 1
    elif res < -1:
        return -1
    else:
        return res #else res should just be -1, 0, or 1 as appropriate.

def runExample():
    m = 3
    n = 3
    Ax = np.ones((m, n))
    Ax[0,1] = 2
    Ax[1,2] = 4
    ###print("Ax:\n", Ax)
    multiPass(Ax, Ax)


def multiPass(Ax, E): # E is initialized as Ax
    m = len(E) # initializing values and empty results lists
    n = E.shape[1]
    ###print(E.shape)
    ###print("m and n:", m, n)
    S = set()
    numiter = 0
    deltas = [] # deltas will always be decreasing
    
    done = False
    while not done: # make this a do while?
        f = np.zeros(n)
        #f = []
        for i in range(n): # is this supposed to be 1 to n? not sure bc of the argmin part.
            # that makes me think list and 0-index, but using a function in general and how it's written implies otherwise.
            ###print("\niter (inner loop up to n): ", i)
            if i in S or np.count_nonzero(E[:,i]) == 0: # 0 vector has been encountered. set f[i]=1 to avoid "singular matrix" error
                f[i] = 1
            else:
                PEiE = getProjection(E, E[:,i])

                #print('\nPEiE:----\n', PEiE)
                #https://stackoverflow.com/questions/9171158/how-do-you-get-the-magnitude-of-a-vector-in-numpy
                resFi = 1 - np.linalg.norm( PEiE ) / np.linalg.norm( E )
                ###print('\nresFi:----\n', resFi)
                f[i] = resFi

        S.add( np.argmin(f) )
        ###print('\nS:----\n', S)

        cols = Ax[:, list(S)] # get all columns with indices in S
        PAxsAx = getProjection(Ax, cols) # this should always have the same dimensions as Ax
        ###print('\nPAxsAx:----\n', PAxsAx)
        ###print('\ncol examination:----\n', E[:,137], PAxsAx[:,137])
        E = Ax - PAxsAx # e.g. 96x999 - 96x999
        ###print('\nnew E:----\n', E[:,137])

        myfunc_vec = np.vectorize(roundToInt)
        Atilde = myfunc_vec(PAxsAx)
        #^https://stackoverflow.com/questions/42594695/how-to-apply-a-function-map-values-of-each-element-in-a-2d-numpy-array-matrix

        Nnz = np.count_nonzero(Ax - Atilde)

        delta = Nnz/(m*n)
        deltas.append(delta) # deltas will always be decreasing
        ###print('\ndeltas:----\n', deltas)
        numiter += 1
        if delta <= .1:
            done = True
    #print('\ndeltas:----\n', deltas)
    #print('\nS:----\n', S)
    return S
    
    
def runActual():
    with open("./matrix_inputs_ax/SORCS3/SORCS3_ACB_encode.json") as f:
        Ax = np.matrix(json.load(f))
    ###print("Ax:\n", Ax)
    return multiPass(Ax, Ax)
    
#runExample()
S = runActual()


In [54]:
#print(str(S) )

{356, 613, 581, 743, 137, 138, 75, 543, 282, 401, 184, 922, 731, 796, 447}


In [60]:
# Actually generate all results (indices of tSNPs) for all populations and regions

# these are all of the populations and regions as in the two files in the CSE549/code/lstruct folder
pops = "ACB,ASW,BEB,CDX,CEU,CHB,CHS,CLM,ESN,FIN,GBR,GIH,GWD,IBS,ITU,JPT,KHV,LWK,MSL,MXL,PEL,PJL,PUR,STU,TSI,YRI"
regions = "SORCS3,17q25,PAH"
pops = pops.split(',')
regions = regions.split(',')


for region in regions:
    for population in pops:
        q = region
        r = population
        print(q,r)
        with open("./matrix_inputs_ax/" + q + "/" + q +'_'+ r +'_encode.json') as f:
            Ax = np.matrix(json.load(f))
        print(Ax.shape)
        S = multiPass(Ax, Ax)
        
        f = open(("./greedy_multipass_results/" + q + "/" + q +'_'+ r + '_multipass.txt'),'w')
        f.write(str(S))
        #for index in S:
            #f.write("%i " % index)
        f.close()

SORCS3 ACB
(96, 999)
SORCS3 ASW
(66, 999)
SORCS3 BEB
(86, 999)
SORCS3 CDX
(99, 999)
SORCS3 CEU
(99, 999)
SORCS3 CHB
(103, 999)
SORCS3 CHS
(108, 999)
SORCS3 CLM
(94, 999)
SORCS3 ESN
(99, 999)
SORCS3 FIN
(99, 999)
SORCS3 GBR
(92, 999)
SORCS3 GIH
(106, 999)
SORCS3 GWD
(113, 999)
SORCS3 IBS
(107, 999)
SORCS3 ITU
(103, 999)
SORCS3 JPT
(104, 999)
SORCS3 KHV
(101, 999)
SORCS3 LWK
(101, 999)
SORCS3 MSL
(85, 999)
SORCS3 MXL
(67, 999)
SORCS3 PEL
(86, 999)
SORCS3 PJL
(96, 999)
SORCS3 PUR
(105, 999)
SORCS3 STU
(103, 999)
SORCS3 TSI
(108, 999)
SORCS3 YRI
(109, 999)
17q25 ACB
(96, 1000)
17q25 ASW
(66, 1000)
17q25 BEB
(86, 1000)
17q25 CDX
(99, 1000)
17q25 CEU
(99, 1000)
17q25 CHB
(103, 1000)
17q25 CHS
(108, 1000)
17q25 CLM
(94, 1000)
17q25 ESN
(99, 1000)
17q25 FIN
(99, 1000)
17q25 GBR
(92, 1000)
17q25 GIH
(106, 1000)
17q25 GWD
(113, 1000)
17q25 IBS
(107, 1000)
17q25 ITU
(103, 1000)
17q25 JPT
(104, 1000)
17q25 KHV
(101, 1000)
17q25 LWK
(101, 1000)
17q25 MSL
(85, 1000)
17q25 MXL
(67, 1000)
17q25 PEL
(8

In [ ]:
print(S)
# Note: the beta_est in getProjection leads to slightly different values for fi than that of getProjection2.

In [19]:
a = -.4
print(int(round(a)) )

#https://stackoverflow.com/questions/10326015/singular-matrix-issue-with-numpy

0


In [85]:
# helpful links
# https://stackoverflow.com/questions/9171158/how-do-you-get-the-magnitude-of-a-vector-in-numpy
# https://stackoverflow.com/questions/8386675/extracting-specific-columns-in-numpy-array
# https://stackoverflow.com/questions/42594695/how-to-apply-a-function-map-values-of-each-element-in-a-2d-numpy-array-matrix

import ast

pops = "ACB,ASW,BEB,CDX,CEU,CHB,CHS,CLM,ESN,FIN,GBR,GIH,GWD,IBS,ITU,JPT,KHV,LWK,MSL,MXL,PEL,PJL,PUR,STU,TSI,YRI"
regions = "SORCS3,17q25,PAH"
pops = pops.split(',')
regions = regions.split(',')


def compare_greedy_results():
    diffRes = []
    print(len(regions), len(pops))
    for region in regions:
        for population in pops:
            q = region
            r = population
            print(q,r)
            with open("./greedy_multipass_results/" + q + "/" + q +'_'+ r + '_multipass.txt') as f:
                with open("./greedy_multipass_results_v2/" + q + "/" + q +'_'+ r +'_tSNPs.json') as f2:
                    indices1 = set(ast.literal_eval(f.read())) # elements are of type int
                    indices2 = set(json.load(f2)) # elements are of type int
                    diff1 = indices1 - indices2
                    diff2 = indices2 - indices1
                    if diff1 or diff2:
                        diffRes.append( (q+r,diff1,diff2) )
                    #print(diff1, ";", diff2)
                
    print("\n--------------\n", diffRes)
    print("done")
    
compare_greedy_results()

3 26
SORCS3 ACB
SORCS3 ASW
SORCS3 BEB
SORCS3 CDX
SORCS3 CEU
SORCS3 CHB
SORCS3 CHS
SORCS3 CLM
SORCS3 ESN
SORCS3 FIN
SORCS3 GBR
SORCS3 GIH
SORCS3 GWD
SORCS3 IBS
SORCS3 ITU
SORCS3 JPT
SORCS3 KHV
SORCS3 LWK
SORCS3 MSL
SORCS3 MXL
SORCS3 PEL
SORCS3 PJL
SORCS3 PUR
SORCS3 STU
SORCS3 TSI
SORCS3 YRI
17q25 ACB
17q25 ASW
17q25 BEB
17q25 CDX
17q25 CEU
17q25 CHB
17q25 CHS
17q25 CLM
17q25 ESN
17q25 FIN
17q25 GBR
17q25 GIH
17q25 GWD
17q25 IBS
17q25 ITU
17q25 JPT
17q25 KHV
17q25 LWK
17q25 MSL
17q25 MXL
17q25 PEL
17q25 PJL
17q25 PUR
17q25 STU
17q25 TSI
17q25 YRI
PAH ACB
PAH ASW
PAH BEB
PAH CDX
PAH CEU
PAH CHB
PAH CHS
PAH CLM
PAH ESN
PAH FIN
PAH GBR
PAH GIH
PAH GWD
PAH IBS
PAH ITU
PAH JPT
PAH KHV
PAH LWK
PAH MSL
PAH MXL
PAH PEL
PAH PJL
PAH PUR
PAH STU
PAH TSI
PAH YRI

--------------
 [('SORCS3BEB', {306}, {346}), ('SORCS3CHB', {346}, {354}), ('SORCS3ESN', set(), {835}), ('SORCS3GBR', {235}, {275}), ('SORCS3ITU', set(), {37}), ('SORCS3PUR', {739}, {818}), ('17q25GIH', set(), {365}), ('17q25PJL', {671}, {6